**Restaurant_id** - идентификационный номер ресторана / сети ресторанов  
**City** - город, в котором находится ресторан  
**Cuisine Style** - кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане  
**Ranking** - место, которое занимает данный ресторан среди всех ресторанов своего города  
**Rating** - рейтинг ресторана по данным TripAdvisor (именно это значение должна была предсказывать модель)  
**Price Range** - диапазон цен в ресторане  
**Number of Reviews** - количетво отзывов в ресторане  
**Reviews** - данные о двух отзывах, которые отображаются на сайте ресторана  
**URL_TA** - URL страницы ресторана на TripAdvisor  
**ID_TA** - идентификатор ресторана в базе данных TripAdvisor

# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd

In [51]:
df0 = pd.read_csv('main_task.csv')
df = df0.copy()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [4]:
# количество пропущенных значений в каждом столбце
df.isnull().sum()

Restaurant_id            0
City                     0
Cuisine Style         9283
Ranking                  0
Rating                   0
Price Range          13886
Number of Reviews     2543
Reviews                  0
URL_TA                   0
ID_TA                    0
dtype: int64

In [5]:
df.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


In [52]:
# удалить столбцы, содержащие данные типа object
# заполнить пропущенные значения нулём или средним арифметическим
#df1 = df.drop([col for col in df.columns if df[col].dtype == 'O'],axis=1)
df1 = df.drop(df.select_dtypes(include=['object']), axis = 1)
df1 = df1.fillna(df1.mean())
df1.head()

,Ranking,Rating,Number of Reviews
0,5570.0,3.5,194.0
1,1537.0,4.0,10.0
2,353.0,4.5,688.0
3,3458.0,5.0,3.0
4,621.0,4.0,84.0


In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ranking            40000 non-null  float64
 1   Rating             40000 non-null  float64
 2   Number of Reviews  40000 non-null  float64
dtypes: float64(3)
memory usage: 937.6 KB


In [8]:
df['City'].dtype

dtype('O')

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [9]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df1.drop('Rating', axis = 1)
y = df1['Rating']

In [10]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [11]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [12]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [13]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [14]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.43134215670024423


In [15]:
# сколько вариантов непустых значений в столбце Price Range
b = df['Price Range'].value_counts()
len(b)

3

In [16]:
# сколько средних значений
b['$$ - $$$']

18412

In [17]:
# сколько городов
len(df['City'].value_counts())

31

In [63]:
# сколько типов кухонь -- разбить и посчитать
df = df0.copy()
#df['Cuisine Style'].value_counts()
df1 = df[['Restaurant_id', 'Cuisine Style']]
df1 = df1.dropna()
df2 = df1.explode('Cuisine Style') # разбиваем по кухням
#df1 = pd.DataFrame(df['Cuisine Style'].values.tolist())
#df1 = pd.get_dummies(df['Cuisine Style'].str.split(','))
a = df2['Cuisine Style'].count()
a
#df2.head()

30717

In [19]:
class ReColumns():
    
    def __init__(self, values):
        # иничиализируем атрибуты
        self.values = values
        # собираем все данные в список
        self.lisst = []
        
    def add_deal(self, values):
        for i in self.values:
            self.lisst.append(i)
    

SyntaxError: unexpected EOF while parsing (<ipython-input-19-c74c2e76a21e>, line 4)

In [ ]:
# средний размер сделки и список клиентов, из которого исключены повторения
# (если заключено несколько сделок с однм и тем же клиентом)
class SalesReport():
    def __init__(self, employee_name):
        self.deals = []
        self.employee_name = employee_name
        
    def add_deal(self, company, amount):
        self.deals.append({'company': company, 'amount': amount})
        
    def total_amount(self):
        return sum([deal['amount'] for deal in self.deals])
    
    def average_deal(self):
        return self.total_amount()/len(self.deals)
    
    def all_companies(self):
        return list(set([deal['company'] for deal in self.deals]))
    
    def print_report(self):
        print("Employee:", self.employee_name)
        print("Total sales:", self.total_amount())
        print("Average sales:", self.average_deal())
        print("Companies:", self.all_companies())
        
report = SalesReport('Ivan Semenov')

report.add_deal("PepsiCo", 120_000)
report.add_deal("SkyEng", 250_000)
report.add_deal("PepsiCo", 20_000)

report.print_report()

In [ ]:
# определить класс IntDataFrame, который принимает список и все числа в этом списке приводит к целым значениям
# метод count, который считает количество ненулевых элементов
# метод unique, который возвращет число уникальных элементов

class IntDataFrame():
    def __init__(self, values):
# инициализируем атрибуты
        self.values = values
# конвертируем числа в целые
        self.to_whole()
    
    def to_whole(self):
        self.values = [int(value) for value in self.values]
        
# подсчтываем количество ненулевых элементов
    def count(self):
        return np.count_nonzero(self.values)
# возвращаем число уникальных элементов
    def unique(self):
        return len(np.unique(self.values))
    
df = IntDataFrame([4.7, 4, 3, 0, 2.4, 0.3, 4])

print(df.count())
print(df.unique())

In [ ]:
# строки приводим к числам, пропуски- заполнить значениями
import statistics

class DataFrame():
    def __init__(self, column, fill_value = 0):
# инициализируем атрибуты
        self.column = column
        self.fill_value = fill_value
# заполнм пропуски
        self.fill_missed()
# конвертируем все элементы в числа
        self.to_float()
    
    def fill_missed(self):
        for i, value in enumerate(self.column):
            if value is None or value == '':
                self.column[i] = self.fill_value
                
    def to_float(self):
        self.column = [float(value) for value in self.column]
        
    def median(self):
        return statistics.median(self.column)
    
    def mean(self):
        return statistics.mean(self.column)
    
    def deviation(self):
        return statistics.stdev(self.column)
    
# воспользуемся классом
df = DataFrame(['1', 17, 4, None, 8])

print(df.column)
print(df.deviation())
print(df.median())